In [ ]:
%matplotlib inline

### torch.nn究竟是什？：What's torch.nn really?

我们建议运行这份教材在notebook而不是script上。下载对应的notebook（.ipynb）文件，点击以下链接。

PyTorch提供优雅设计的模块和类
`torch.nn <https://pytorch.org/docs/stable/nn.html>`_ ,
`torch.optim <https://pytorch.org/docs/stable/optim.html>`_ ,
`Dataset <https://pytorch.org/docs/stable/data.html?highlight=dataset#torch.utils.data.Dataset>`_ ,
and `DataLoader <https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader>`_
来帮助你创建和调整神经网络。

为了完全地利用他们的力量以及定制化地解决你的问题，你需要真的理解我们在做什么。为了更好地理解，我们会首先在mnist数据集上训练基础的神经网络。为了建立这种理解，我们会功能性地使用最基本的PyTorch tensor。然后，我们会一次次逐渐增加特性从"torch.nn","torch.optim","Dataset","DataLoader"，展示每一块是做什么的，以及它是怎么使得代码要么更简介，要么更灵活。

**这份教材假定你已经安装好了PyTorch，以及熟悉了基本的tensor操作。**
（如果你已经熟悉了numpy数组操作，你会发现PyTorch tensor操作是差不多了。


MNIST data setup
----------------
我们会使用经典的`MNIST <http://deeplearning.net/data/mnist/>`_数据集，这里面包含了手写数字0到9的黑白数据集。

我们会使用 `pathlib <https://docs.python.org/3/library/pathlib.html>`_来处理路径(部分python3的标准库)，然后会使用`requests <http://docs.python-requests.org/en/master/>`_来下载数据。我们只会在我们使用模块时import它，所以你可以看到在每一步它们是怎么被使用的。


In [ ]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

This dataset is in numpy array format, and has been stored using pickle,
a python-specific format for serializing data.



In [ ]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

Each image is 28 x 28, and is being stored as a flattened row of length
784 (=28x28). Let's take a look at one; we need to reshape it to 2d
first.



In [ ]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

PyTorch 使用 ``torch.tensor``, 而不是numpy数组，所以我们需要去转化我们的数据。

In [ ]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

Neural net from scratch (no torch.nn)
---------------------------------------------

让我们首先使用PyTorch tensor操作创建一个模型。我们假定你已经熟悉神经网络的基本操作。（如果你不熟悉，你可以学习它们`course.fast.ai <https://course.fast.ai>`_)

PyTorch 提供了创建随机或者zero-filled tensor，那里我们会为简单线性模型创建我们的权重和误差。这些只是常规tensor，带着一个非常特殊的功能：我们告诉pytorch他们需要一个梯度下降。这造成了pytorch记录所有在tensor上的操作，这样他就可以在自动反向传播中计算梯度下降。

对于权重，我们在初始化之后设置了"requires_grad"，这样我们不会想包括了梯度下降这步。（注意到trailling"_"在pytorch中指定in-place表现的操作。）

<div class="alert alert-info"><h4>Note</h4><p>我们用这个来初始化权重`Xavier initialisation <http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf>`_
   (by multiplying with 1/sqrt(n)).</p></div>

In [ ]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

感谢pytorch自动计算梯度下降的能力，我们可以像模型一样使用任何标准的python函数（或者callable的对象）！所以让只是写一个plain矩阵相乘和broadcasted addition来创建一个简单的线性模型。我们同时需要一个激活函数，这样我们会写`log_softmax`以及使用它。记得：虽然PyTorch提供很多预写好损失函数，激活函数等等，你可以使用plain来简单地写你自己的。PyTorch会甚至创建更快的GPU或者vectorized CPU代码给你函数自动化。

In [ ]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

在以上的例子里，``@``代表点乘运算，我们会call我们的函数生气每一个batch的数据(在这里，64 images)。这就是一个*forward pass*。注意到我们预测在这个阶段不会比随机预测准，因为我们就是从随机权重开始的。

In [ ]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

在我们看来，那个``preds``的tensor包含不只是tensor值，还有一个梯度下降函数。我们之后在反向传播中使用这个。

让我们使用负log-likelihood来作为损失函数使用(再说一遍，我们可以只是使用标准python):


In [ ]:
def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

让我们检查我们随机模型的损失，这样我们可以看我们是不是在反向传播之后提高了效果。

In [ ]:
yb = y_train[0:bs]
print(loss_func(preds, yb))

让我们同时使用函数来计算模型的准确率。对于每一个预测，如果最大值带有的index与目标值匹配，那么预测值是正确的。


In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

让我们检查我们随机模型的准确性，这样我们可以看到我们的准确率是不是随着我们的损失提高。


In [ ]:
print(accuracy(preds, yb))

我们可以现在运行一个训练循环。对于每一次迭代，我们会:

- 选择一个数据的mini-batch(大小是 ``bs``)
- 使用模型来得到预测
- 计算损失
- ``loss.backward()`` 更新模型的梯度下降，在这个案例里，是 ``weights``和``bias``。

我们现在使用这些梯度下降来更新权重和误差。我们通过使用``torch.no_grad()``文本管理来操作，因为我们不想这些行动来记录我们下一次梯度下降的计算。你可以看PyTorch's 自动梯度记录操作`here <https://pytorch.org/docs/stable/notes/autograd.html>`_。然后我们设置梯度下降到零，这样我们就准备好下一个循环了。否则，我们梯度下降会记录一个所有所发生操作的运行的tally（比如说``loss.backward()``*adds*那个梯度下降到所有被储存的，而不是替代它们）。

.. tip:: 你可以使用标准的python来debugger到PyTorch的代码，允许你来检查每一步的各种变量值。取消下面的``set_trace()`` 语句来尝试。


In [ ]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        #         set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()

这就是：我们从无到有地创建和训练了一个最小神经网络(在这里，是一个逻辑回归模型，因为我们没有隐藏层)。

让我们检查损失和准确度和跟我们之前所有的东西比较。我们期待损失会下降以及准确率会上升，它们确实是这样的。


In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Using torch.nn.functional
------------------------------

我们现在会重构我们的代码，这样它就跟之前一样，只有我们开始利用PyTorch ``nn`` 的类来让它更加简洁和灵活。在这里的每一步，我们都应该让我们的代码多多少少地：更短，可读性更强，以及更灵活。

第一步和更简单地一步是通过替代那些从``torch.nn.functional``得到的我们的手写激活函数和损失函数去让我们的代码更短(那些通常从名字空间``F``中import进来的)。这个模块包括所有在``torch.nn`` 库里的函数(然而在类里包含库的其他部分)。同时，一个更广阔的损失和激活函数，你可以同时找到一些传统创建神经网络的函数，比如pooling函数。(这里同时有传统的卷积，线性层，等等，但是正如我们所看到的，这里通常会更好地使用库里的其他部分来处理。)

如果你使用负log likelihood损失和log softmax激活函数，那么Pytorch提供了一个简单的函数``F.cross_entropy``，它结合了上面两个。这样我们可以甚至把激活函数踢出我们的模型。


In [ ]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

注意到我们不再在``model``函数里call``log_softmax``了。让我们确认我们的损失和准确度是跟之前一样的:


In [ ]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

Refactor using nn.Module
-----------------------------
接下来，我们会使用``nn.Module``和``nn.Parameter``，对于一个明确和更加简洁的训练循环。我们子类``nn.Module``(他自己是一个类和足够去跟踪状态)。在这个案例里，我们想去创建一个类，来保持住我们的权重，误差，以及前一步的方法。``nn.Module`` 有很多特性和方法(例如``.parameters()``和``.zero_grad()``)这些我们会使用的。


<div class="alert alert-info"><h4>Note</h4><p>``nn.Module`` (大写的M) 是一个PyTorch中专门的概念，也是一个我们会经常使用的类。
 ``nn.Module`` 是跟Python概念不一样的(小写的 ``m``) `模块 <https://docs.python.org/3/tutorial/modules.html>`_,这里是可以被导入的Python代码。</p></div>

In [ ]:
from torch import nn

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb @ self.weights + self.bias

既然我们现在使用了对象而不是只是函数，我们要用模型的例子来说明:

In [ ]:
model = Mnist_Logistic()

现在我们可以用之前一样的方法来计算损失。注意到``nn.Module``对象被使用就像他们是函数一样(因为他们是*callable*)，但是这之后的场景是Pytorch会自动地呼叫我们的``forward``方法。

In [ ]:
print(loss_func(model(xb), yb))

之前为了我们训练循环我们需要按照名字更新每一组参数，然后手动单独地为每一个参数归零的梯度下降，像这样:
::

  with torch.no_grad():
      weights -= weights.grad * lr
      bias -= bias.grad * lr
      weights.grad.zero_()
      bias.grad.zero_()
现在我们可以利用model.parameters()和model.zero_grad() (这些都是通过``nn.Module``被PyTorch定义的)，特别地如果我们有更复杂的模型:::


  with torch.no_grad():
      for p in model.parameters(): p -= p.grad * lr
      model.zero_grad()

我们会圆满地完成在``fit``函数中每一个小训练循环，这样我们可以一次又一次地运行它们。

In [ ]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

fit()

让我们再次检查我们的损失是否变小:

In [ ]:
print(loss_func(model(xb), yb))

Refactor using nn.Linear
-------------------------


我们继续重组我们的代码。不是人工的定义和初始化``self.weights``和``self.bias``，以及计算``xb  @
self.weights + self.bias``，我们会使用Pytorch类`nn.Linear <https://pytorch.org/docs/stable/nn.html#linear-layers>`_ 来为一个线性层，它为我们做所有的东西。Pytorch有很多种预训练层是可以很大地简化我们代码，以及常常加快我们的程序。

In [ ]:
class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)

    def forward(self, xb):
        return self.lin(xb)

我们案例化我们的模型以及用相同的方式计算损失:

In [ ]:
model = Mnist_Logistic()
print(loss_func(model(xb), yb))

我们仍然可以使用跟之前相同的``fit``方法。

In [ ]:
fit()

print(loss_func(model(xb), yb))

Refactor using optim
------------------------------

Pytorch同时有各种各样优化算法的库``torch.optim``。我们可以从我们的优化器来使用``step``方法来做向前一步，而不是手动的更新每一个参数。

这个会让我们替代我们原先手动的代码优化步骤:
::

  with torch.no_grad():
      for p in model.parameters(): p -= p.grad * lr
      model.zero_grad()

相反地我们只是使用:
::
  opt.step()
  opt.zero_grad()

(``optim.zero_grad()`` 重新设置梯度到0而且我们需要在计算出梯度之前call它来进行下一组minibatch。)

In [ ]:
from torch import optim

我们会定义一个小函数来创建我们的模型和优化器，这样我们可以在未来重复使用它。


In [ ]:
def get_model():
    model = Mnist_Logistic()
    return model, optim.SGD(model.parameters(), lr=lr)

model, opt = get_model()
print(loss_func(model(xb), yb))

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

Refactor using Dataset
------------------------------

PyTorch有一个概括性的数据集类。一个数据集可以被所有的
has an abstract Dataset class.  A Dataset can be anything that has
a ``__len__`` function (called by Python's standard ``len`` function) and
a ``__getitem__`` function as a way of indexing into it.


`这份教程 <https://pytorch.org/tutorials/beginner/data_loading_tutorial.html>`_通过一个很好的创建传统``FacialLandmarkDataset``类作为``Dataset``数据集的子类。


PyTorch's `TensorDataset <https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset>`_是一个有包装器tensor的数据集。通过定义一个长度和indexing的方式，这个同时会给到我们一种循环、索引和在tensor第一纬度切片的方式，这个会让它更加简单来启动独立和非独立变量在训练时相同的line。


In [ ]:
from torch.utils.data import TensorDataset

``x_train``和``y_train`` 可以结合到一个独立的``TensorDataset``,这会使得在切片中循环变得更简单。


In [ ]:
train_ds = TensorDataset(x_train, y_train)

之前的，我们有单独为x和y值遍历的minibatch:
::
    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]

现在，我们可以通过这两步一起:
::
    xb,yb = train_ds[i*bs : i*bs+bs]




In [ ]:
model, opt = get_model()

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        xb, yb = train_ds[i * bs: i * bs + bs]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

Refactor using DataLoader
------------------------------

Pytorch's ``DataLoader``是负责batch的。你可以从任何的``Dataset``来创建一个``DataLoader``。``DataLoader``让它更容易地去遍历它每一个batches。而不是必须去使用``train_ds[i*bs : i*bs+bs]``，DataLoader自动地给我们每一个minibatch。


In [ ]:
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)

之前，我们在batches(xb,yb)上的loop循环像以下般:
::
      for i in range((n-1)//bs + 1):
          xb,yb = train_ds[i*bs : i*bs+bs]
          pred = model(xb)

现在，我们的loop循环是更干净的，正如 (xb, yb)是自动加载到data loader一样:
::
      for xb,yb in train_dl:
          pred = model(xb)



In [ ]:
model, opt = get_model()

for epoch in range(epochs):
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

多亏Pytorch的``nn.Module``, ``nn.Parameter``, ``Dataset``,以及``DataLoader``,我们训练的循环是很奇妙地更小或者更简单地去理解。让我们现在尝试去增加必须的基础特征来创建实用的模型。


Add validation
-----------------------

在section 1中，我们只是去尝试一个合理的训练循环来建立训练数据集。事实上，你**总是** 应该一个`validation set <https://www.fast.ai/2017/11/13/validation-sets/>`_,为了识别你是不是过拟合。

打乱抽取的训练数据是`important <https://www.quora.com/Does-the-order-of-training-data-matter-when-training-neural-networks>`_为了防止batches和overfitting的相关性。另一方面，验证损失会和我们是否打乱测试集保持一致。因为无故打乱是需要额外的时间的，所以无端打乱测试集是不合理的。

我们会使用比训练集大一倍的一个batch size测试集。这是因为测试集不需要反向传播所以需要更少的记忆(这不需要储存梯度)。我们会利用这个来使用更大的batch size和更快地计算损失。


In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

我们会在每一个epoch后计算以及打印测试集损失。

(注意到我们总是在训练前呼叫``model.train()``，以及在预测前呼叫``model.eval()``，因为这些是被诸如``nn.BatchNorm2d``的层所使用，以及 ``nn.Dropout``来保证不同阶段的合适的行为。)

In [ ]:
model, opt = get_model()

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(model(xb), yb) for xb, yb in valid_dl)

    print(epoch, valid_loss / len(valid_dl))

Create fit() and get_data()
----------------------------------
我们会做小小的重构。因为我们经过了为每一个训练集和测试集计算损失的过程两次，让我们把它写进函数， ``loss_batch``，这样会帮我们计算每一个batch的损失。

我们通过每一个训练集的优化器，以及使用它来得到反向传播。对于测试集，我们不会传递优化器，这样的方法不会实现方向传播。


In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

``fit`` 运行必要的操作来训练我们的模型以及为每一个epoch计算训练和测试集的损失。

In [ ]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

``get_data`` 返回训练集和测试集的dataloaders。

In [ ]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

现在，我们整个过程都是获取loaders和fitting模型，我们可以通过三行代码实现:

In [ ]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

你可以使用这基础的三行代码来训练各种各样的模型。
让我们看下我们怎么用它们来训练卷积神经网络(CNN)!

Switch to CNN
-------------
我们现在准备用三个卷积层来建立我们的神经网络。因为之前没有这样假定任何模型形式的函数，我们能够不做任何修改地使用它们来训练CNN。

我们会使用Pytorch的预定义
`Conv2d <https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d>`_ 类作为我们的卷基层。我们用三层卷基层来定义CNN。
每一个卷积层都接着ReLU。在最后，我们实现一个average pooling。(注意到``view``是PyTorch版本的numpy``reshape``)

In [ ]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

`Momentum <https://cs231n.github.io/neural-networks-3/#sgd>`_是一个随机梯度下降的变化率，这让之前的更新都即入内，它会逐渐地使训练过程变得越来越快。

In [ ]:
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

nn.Sequential
------------------------

``torch.nn`` 有另外一个更加有用的类我们可以使用来简化我们的代码:`Sequential <https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential>`_ 。
一个``Sequential`` 对象在运行每一个在它里面的模块时，都是按顺序的。这是写我们神经网络简单的方法。

为了利用这个，我们需要通过一个给定的函数简单地定义一个**custom layer**。例如，PyTorch没有`view`层，我们需要为我们的神经网络创建一个。
``Lambda``会创建一个我们可以使用的用``Sequential``定义神经网络层。

In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def preprocess(x):
    return x.view(-1, 1, 28, 28)

这个模型创建了简单的``Sequential``:

In [ ]:
model = nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

Wrapping DataLoader
-----------------------------

Our CNN is fairly concise, but it only works with MNIST, because:
 - It assumes the input is a 28\*28 long vector
 - It assumes that the final CNN grid size is 4\*4 (since that's the average
pooling kernel size we used)

Let's get rid of these two assumptions, so our model works with any 2d
single channel image. First, we can remove the initial Lambda layer but
moving the data preprocessing into a generator:



In [ ]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Next, we can replace ``nn.AvgPool2d`` with ``nn.AdaptiveAvgPool2d``, which
allows us to define the size of the *output* tensor we want, rather than
the *input* tensor we have. As a result, our model will work with any
size input.



In [ ]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Let's try it out:



In [ ]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

Using your GPU
---------------

If you're lucky enough to have access to a CUDA-capable GPU (you can
rent one for about $0.50/hour from most cloud providers) you can
use it to speed up your code. First check that your GPU is working in
Pytorch:



In [ ]:
print(torch.cuda.is_available())

And then create a device object for it:



In [ ]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

Let's update ``preprocess`` to move batches to the GPU:



In [ ]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)


train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

Finally, we can move our model to the GPU.



In [ ]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

You should find it runs faster now:



In [ ]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

Closing thoughts
-----------------

现在你有了数据pipeline和很多种pytorch模型的训练循环。来看看简单的模型训练可以如何实现，请看'mnist_sample'的样本notebook。

当然，你可以往里面添加很多东西，例如数据增强，调参数，监控训练过程，迁移学习等等。这些特性在fastai中都可以使用，这可以跟这份教材一样使用相同的训练方法，为使用者进一步提高他们的模型提供下一步。

我们在这份教材一开始都承诺了我们会解释"torch.nn","torch optim","Dataset"和"DataLoader"的例子。因此让我们一起来总结我们所看过的：



 - **torch.nn**

   + ``Module``: 创建一个表现得像是函数的callable，但是可以同时包含state（比如说神经网络层权重）。它知道自己包含了什么"Parameter"(s)，而且可以把它所有的梯度归零，不断地循环来进行权重更新等等。
   + ``Parameter``: 一个tensor的装饰器，它告诉"Module"在backprop时它用权重需要升级。只有带有"requires_grad"的tensor特性设置是更新的。   
   + ``functional``: 一个模块（通常import进"F"名字空间）其中包括激活函数，损失函数等等，同时non-stateful版本的layers例如卷积和线性层。
 - ``torch.optim``: 包含例如是"SGD"的优化器，其中在backward步骤中更新了"Parameter"的权重。
 - ``Dataset``:一个带有"__len__"和"__getitem__"的简要的对象界面，包括用Pytorch提供的例如是"TensorDataset"的类／
 - ``DataLoader``: 拿任意"Dataset"和创建一个返回batchs of data的循环器。
 